In [1]:
!jt -t oceans16

# Importacion del dataset

El primer paso para el preprocesamiento de datos es la importacion del dataset

In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model, model_selection

df = pd.read_csv("datosfiltrados.csv", sep=",",index_col=False)

df.head()

,ix,Id,danho,ciclo,Cod.Asign,Asignatura,Cod.Car.Sec,Cod.Curso,Convocatoria,Anho,Semestre,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final,id_anony
0,135020,503,2017,1,13008,CALCULO 2,ELE-PLS13,2,1,2017,1,N,0,7,0,0,0,0,NaN,es_1444
1,135021,504,2017,1,13008,CALCULO 2,CGF-PLS13,2,1,2017,1,N,0,5,0,0,0,0,NaN,es_4238
2,135022,505,2017,1,13008,CALCULO 2,ELE-PLS13,2,1,2017,1,S,2016,0,0,0,0,0,"1F-1,2F-2",es_4245
3,135023,506,2017,1,13008,CALCULO 2,CIV-PLS13,2,1,2017,1,N,0,0,0,0,0,0,NaN,es_4967
4,135024,507,2017,1,13008,CALCULO 2,CGF-PLS13,2,1,2017,1,S,2017,21,10,0,0,0,3F-C-3,es_414


# Discriminacion de datos innecesarios

Para esta ocasion, voy a utilizar los datos de todas las asignaturas, no solo la de los alumnos de probabilidad y estadistica. Sin embargo, aun es necesario eliminar ciertos parametros. Para mi dataset final, busco tener como datos la asignatura, los puntajes del primer parcial, el taller, el anho, el ciclo y si tiene o no firma el alumno

Primero voy a verificar el porcentaje de datos no disponibles de cada columna del dataset

In [3]:
#Con este comando busco los datos no disponibles
df.isna().sum()

#Con esta funcion determino el porcentaje de datos no disponible
df.isna().sum()/len(df)*100

ix               0.000000
Id               0.000000
danho            0.000000
ciclo            0.000000
Cod.Asign        0.000000
Asignatura       0.000000
Cod.Car.Sec      0.000000
Cod.Curso        0.000000
Convocatoria     0.000000
Anho             0.000000
Semestre         0.000000
Aprobado         0.000000
Anho.Firma       0.000000
Primer.Par       0.000000
Segundo.Par      0.000000
AOT              0.000000
Primer.Rec       0.000000
Segundo.Rec      0.000000
Nota.Final      48.855091
id_anony         0.000000
dtype: float64

Primeramente, los datos de la asignatura, el ciclo, el anho, los datos del primer parcial y del segundo parcial estan completos. Eliminaremos las columnas de datos que no nos sirven para conservar unicamente los datos que son relevantes para este proyecto

In [4]:
#Con esta funcion, recorto las columnas que no necesito utilizar

df=df.drop(['Semestre','ix','Id','Cod.Car.Sec','Cod.Curso'],axis=1)

df.head()

,danho,ciclo,Cod.Asign,Asignatura,Convocatoria,Anho,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final,id_anony
0,2017,1,13008,CALCULO 2,1,2017,N,0,7,0,0,0,0,NaN,es_1444
1,2017,1,13008,CALCULO 2,1,2017,N,0,5,0,0,0,0,NaN,es_4238
2,2017,1,13008,CALCULO 2,1,2017,S,2016,0,0,0,0,0,"1F-1,2F-2",es_4245
3,2017,1,13008,CALCULO 2,1,2017,N,0,0,0,0,0,0,NaN,es_4967
4,2017,1,13008,CALCULO 2,1,2017,S,2017,21,10,0,0,0,3F-C-3,es_414


Yo necesito conservar el dato el anho y el ciclo. Pero en el dataset tengo las columnas danho,ciclo,Convocatoria y Anho. Voy a conservar el ciclo y eliminar la convocatoria. Sin embargo, es necesario saber si danho y Anho son lo mismo

In [5]:
if df["danho"].all()==df["Anho"].all():
    print("Contienen la misma informacion")

Contienen la misma informacion


In [6]:
#Ahora que se que tienen la misma informacion, eliminare las columnas innecesarias
df=df.drop(['Convocatoria','Anho'],axis=1)

#Tambien es necesario renombrar las columnas a algo mas identificable
df.rename(columns={'danho': 'Anho','ciclo' : 'Ciclo'},inplace = True)

# Preprocesamiento de datos
Aqui es donde empieza lo complicado, va a ser necesario modificar el dataset para poder utilizar ciertos datos

Primero vamos a trabajar con los alumnos que tienen firma del semestre pasado. Tenemos como dato la columna "Anho.Firma". Esta nos indica si es que el alumno tiene o no una firma de un ciclo anterior. En caso positivo, indica de que anho es dicha firma. Los casos negativos se representan con un cero. El anho donde el alumno consiguio firma no es importante para nuestro analisis, solo es importante saber si la tiene o no. Por este motivo, en el siguiente bloque de codigo reemplazar esta columna por otra que sea de utilidad

In [7]:
#Primero hago una comparacion para verificar si cada dato de Anho.Firma es diferente de cero
firma_anterior=df['Anho.Firma']!=0
#Ahora tenemos un vector con true o false dependiendo si el alumno tenia firma o no de un semestre pasado. Con este vector, se
#puede reemplazar Anho.Firma por una nueva columna que tenga como datos a este vector

#Primero insertamos la nueva columna
df.insert(9,"RecursaConFirma",firma_anterior)
df["RecursaConFirma"].replace({True: '1', False: '0'}, inplace=True)

#Ahora eliminamos la columna de Anho.Firma
df=df.drop(['Anho.Firma'],axis=1)

df.head()

,Anho,Ciclo,Cod.Asign,Asignatura,Aprobado,Primer.Par,Segundo.Par,AOT,RecursaConFirma,Primer.Rec,Segundo.Rec,Nota.Final,id_anony
0,2017,1,13008,CALCULO 2,N,7,0,0,0,0,0,NaN,es_1444
1,2017,1,13008,CALCULO 2,N,5,0,0,0,0,0,NaN,es_4238
2,2017,1,13008,CALCULO 2,S,0,0,0,1,0,0,"1F-1,2F-2",es_4245
3,2017,1,13008,CALCULO 2,N,0,0,0,0,0,0,NaN,es_4967
4,2017,1,13008,CALCULO 2,S,21,10,0,1,0,0,3F-C-3,es_414


Ahora que terminamos de clasificar las firmas de los recursantes, vamos a organizar las firmas de los que estan cursando la materia. Empezaremos por renombrar las columnas de los parciales y talleres por algo mas facil de identificar

In [8]:
#Con esta funcion renombramos las columnas de datos
df.rename(columns={'AOT': 'Taller','Primer.Par' : '1P','Segundo.Par':'2P'},inplace = True)
df.head()

,Anho,Ciclo,Cod.Asign,Asignatura,Aprobado,1P,2P,Taller,RecursaConFirma,Primer.Rec,Segundo.Rec,Nota.Final,id_anony
0,2017,1,13008,CALCULO 2,N,7,0,0,0,0,0,NaN,es_1444
1,2017,1,13008,CALCULO 2,N,5,0,0,0,0,0,NaN,es_4238
2,2017,1,13008,CALCULO 2,S,0,0,0,1,0,0,"1F-1,2F-2",es_4245
3,2017,1,13008,CALCULO 2,N,0,0,0,0,0,0,NaN,es_4967
4,2017,1,13008,CALCULO 2,S,21,10,0,1,0,0,3F-C-3,es_414


Ahora es necesario verificar si el alumno tiene firma para rendir los examenes finales. La firma se divide en las siguientes categorias:
<ul>
<li>Firma para recu: La suma de 1P,2P y taller del alumno alcanzan los 15 puntos</li>
<li>Firma para final: El alumno es un recursante con firma o la suma del 1P,2P y taller >= 35 puntos</li>
</ul>

Un pequenho problema que tendra este filtrado es que no tendra en cuenta el requisito minimo de taller de la materia. Es decir, si el alumno tiene 20 en el 1P, 15 en el 2P y 2 en el taller, cuando la materia pide como minimo 6, este alumno entrara en el grupo de alumnos que "tienen" firma y va a poder rendir tanto el final como el recuperatorio de la materia.


In [9]:
#Primero hallamos la firma que tiene cada alumno
firma=df["1P"]+df["2P"]+df["Taller"]

#Verificamos si el alumno tiene el puntaje para habilitar recu
firmaR=firma>=15

#Verificamos si el alumno tiene el puntaje para habilitar recu
firmaF=firma>=35

#Aqui identificamos si cada alumno tiene derecho a rendir recuperatorio y/o final
#OBS: Se tiene en cuenta que los recursantes con firma pueden rendir ambos
Drecu=np.logical_or(firmaR,df["RecursaConFirma"])
Dfinal=np.logical_or(firmaF,df["RecursaConFirma"])

#Ahora insertamos en el dataframe los datos adquiridos en este analisis
df.insert(10,"DerechoAFinal",Dfinal)
df.insert(10,"DerechoARecuperatorio",Drecu)

#Reemplazamos si tienen derecho o no por '1' y '0'
df["DerechoAFinal"].replace({True: '1', False: '0'}, inplace=True)
df["DerechoARecuperatorio"].replace({True: '1', False: '0'}, inplace=True)

df.head()

,Anho,Ciclo,Cod.Asign,Asignatura,Aprobado,1P,2P,Taller,RecursaConFirma,Primer.Rec,DerechoARecuperatorio,DerechoAFinal,Segundo.Rec,Nota.Final,id_anony
0,2017,1,13008,CALCULO 2,N,7,0,0,0,0,0,0,0,NaN,es_1444
1,2017,1,13008,CALCULO 2,N,5,0,0,0,0,0,0,0,NaN,es_4238
2,2017,1,13008,CALCULO 2,S,0,0,0,1,0,1,1,0,"1F-1,2F-2",es_4245
3,2017,1,13008,CALCULO 2,N,0,0,0,0,0,0,0,0,NaN,es_4967
4,2017,1,13008,CALCULO 2,S,21,10,0,1,0,1,1,0,3F-C-3,es_414


Todo alumno que no alcanza el derecho a examen recuperatorio recursa la materia. Este dato no es util para nuestro analisis. Por ende, eliminaremos a todos los alumnos que no cumplen con dicho requisito

In [10]:
df=df[df.DerechoARecuperatorio=='1']

#Como la columna de DerechoARecuperatorio ya no tiene datos significativos para nuestro analisis, podemos eliminarla
df=df.drop(['DerechoARecuperatorio'],axis=1)

df.head()

,Anho,Ciclo,Cod.Asign,Asignatura,Aprobado,1P,2P,Taller,RecursaConFirma,Primer.Rec,DerechoAFinal,Segundo.Rec,Nota.Final,id_anony
2,2017,1,13008,CALCULO 2,S,0,0,0,1,0,1,0,"1F-1,2F-2",es_4245
4,2017,1,13008,CALCULO 2,S,21,10,0,1,0,1,0,3F-C-3,es_414
7,2017,1,13008,CALCULO 2,N,19,0,0,0,0,0,0,NaN,es_4556
9,2017,1,13008,CALCULO 2,S,0,20,0,1,0,1,0,"2F-C-1,3F-C-3",es_4971
10,2017,1,13008,CALCULO 2,S,13,16,0,1,0,1,0,1F-C-3,es_4972


In [11]:
#Finalmente, reemplazamos las S y N de Aprobado por True y False para manipular con mayor facilidad los datos de esa columna
df["Aprobado"].replace({"S": '1', "N": '0'}, inplace=True)
display(df)

,Anho,Ciclo,Cod.Asign,Asignatura,Aprobado,1P,2P,Taller,RecursaConFirma,Primer.Rec,DerechoAFinal,Segundo.Rec,Nota.Final,id_anony
2,2017,1,13008,CALCULO 2,1,0,0,0,1,0,1,0,"1F-1,2F-2",es_4245
4,2017,1,13008,CALCULO 2,1,21,10,0,1,0,1,0,3F-C-3,es_414
7,2017,1,13008,CALCULO 2,0,19,0,0,0,0,0,0,NaN,es_4556
9,2017,1,13008,CALCULO 2,1,0,20,0,1,0,1,0,"2F-C-1,3F-C-3",es_4971
10,2017,1,13008,CALCULO 2,1,13,16,0,1,0,1,0,1F-C-3,es_4972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17682,2019,2,13018,DINAMICA,1,19,29,10,1,0,1,0,1F-5,es_7090
17683,2019,2,13018,DINAMICA,1,0,0,0,1,0,1,0,1F-4,es_8362
17684,2019,2,13018,DINAMICA,1,20,28,7,1,0,1,0,1F-5,es_7715
17685,2019,2,13018,DINAMICA,0,19,20,5,1,0,1,0,NaN,es_7095


Finalmente, como un extra, vamos a extraer la calificacion final de cada alumno

In [12]:
#Aqui creo una columna en donde se asigna con que puntaje paso
df.assign(Calificacion = 0)

#Esta bandera contiene a todos los que tienen NaN en sus casillas
flagNA=df["Nota.Final"].isna()

#Esta bandera contiene a todos los que NO tienen NaN en sus casillas
flagNotNA=np.logical_not(flagNA)


#Esta linea de codigo asigna la nota final de cada alumno de la catedra
df.loc[flagNotNA,"Calificacion"]=df["Nota.Final"][flagNotNA].str.split(',').str[-1].str.split('-').str[-1].astype('int')

#Finalmente reemplazaremos los espacios NaN de algunas columnas para facilitar el trabajo en otras librerias
values={'Calificacion':0}
df.fillna(value=values,inplace=True)

df.head()

,Anho,Ciclo,Cod.Asign,Asignatura,Aprobado,1P,2P,Taller,RecursaConFirma,Primer.Rec,DerechoAFinal,Segundo.Rec,Nota.Final,id_anony,Calificacion
2,2017,1,13008,CALCULO 2,1,0,0,0,1,0,1,0,"1F-1,2F-2",es_4245,2.0
4,2017,1,13008,CALCULO 2,1,21,10,0,1,0,1,0,3F-C-3,es_414,3.0
7,2017,1,13008,CALCULO 2,0,19,0,0,0,0,0,0,NaN,es_4556,0.0
9,2017,1,13008,CALCULO 2,1,0,20,0,1,0,1,0,"2F-C-1,3F-C-3",es_4971,3.0
10,2017,1,13008,CALCULO 2,1,13,16,0,1,0,1,0,1F-C-3,es_4972,3.0


In [13]:
#Ahora podemos eliminar la columna de Nota.Final
df=df.drop(['Nota.Final'],axis=1)

display(df)

,Anho,Ciclo,Cod.Asign,Asignatura,Aprobado,1P,2P,Taller,RecursaConFirma,Primer.Rec,DerechoAFinal,Segundo.Rec,id_anony,Calificacion
2,2017,1,13008,CALCULO 2,1,0,0,0,1,0,1,0,es_4245,2.0
4,2017,1,13008,CALCULO 2,1,21,10,0,1,0,1,0,es_414,3.0
7,2017,1,13008,CALCULO 2,0,19,0,0,0,0,0,0,es_4556,0.0
9,2017,1,13008,CALCULO 2,1,0,20,0,1,0,1,0,es_4971,3.0
10,2017,1,13008,CALCULO 2,1,13,16,0,1,0,1,0,es_4972,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17682,2019,2,13018,DINAMICA,1,19,29,10,1,0,1,0,es_7090,5.0
17683,2019,2,13018,DINAMICA,1,0,0,0,1,0,1,0,es_8362,4.0
17684,2019,2,13018,DINAMICA,1,20,28,7,1,0,1,0,es_7715,5.0
17685,2019,2,13018,DINAMICA,0,19,20,5,1,0,1,0,es_7095,0.0


# Exportacion
Ahora que ya terminamos el procesamiento de los datos. Lo unico que queda por hacer es exportar el dataframe para utilizarlo en futuros proyectos

In [14]:
df.to_csv("DFResumido.csv")

In [15]:
#Tener en cuenta que hace falta una forma de discriminar a los alumnos que recursan con firma y no pasan
#Aun es necesario discriminar si tienen o no el puntaje minimo de taller